In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
%cd /content/drive/MyDrive/github/LightGCN_PyTorch/

/content/drive/MyDrive/github/LightGCN_PyTorch


In [5]:
pip install -r requirements.txt

ERROR: Could not find a version that satisfies the requirement torch==1.4.0 (from versions: 1.11.0, 1.12.0, 1.12.1, 1.13.0, 1.13.1, 2.0.0, 2.0.1)
ERROR: No matching distribution found for torch==1.4.0


In [6]:
%cd /content/drive/MyDrive/github/LightGCN_PyTorch/code

/content/drive/MyDrive/github/LightGCN_PyTorch/code


In [38]:
#path = '/content/drive/MyDrive/github/LightGCN/Data/gowalla'
#dataset_gowalla = dataloader.Loader(config={'A_split': True, 'A_n_fold': 5}, path=path)

loading [/content/drive/MyDrive/github/LightGCN/Data/gowalla]
810128 interactions for training
217242 interactions for testing
gowalla Sparsity : 0.0008396216228570436
gowalla is ready to go


In [8]:
pip install tensorboardX

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.6/101.6 kB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 304.5/304.5 kB 29.2 MB/s eta 0:00:00
  Attempting uninstall: protobuf
    Found existing installation: protobuf 3.20.3
    Uninstalling protobuf-3.20.3:
      Successfully uninstalled protobuf-3.20.3


In [9]:
%run main.py --decay=1e-4 --lr=0.001 --layer=3 --seed=2020 --dataset="yelp2018" --topks="[20]" --recdim=64 --epochs=100

>>SEED: 2020
loading [../data/yelp2018]
1237259 interactions for training
324147 interactions for testing
yelp2018 Sparsity : 0.0012958757851778647
yelp2018 is ready to go
===========config================
{'A_n_fold': 100,
 'A_split': False,
 'bigdata': False,
 'bpr_batch_size': 2048,
 'decay': 0.0001,
 'dropout': 0,
 'keep_prob': 0.6,
 'latent_dim_rec': 64,
 'lightGCN_n_layers': 3,
 'lr': 0.001,
 'multicore': 0,
 'pretrain': 0,
 'test_u_batch_size': 100}
cores for test: 1
comment: lgn
tensorboard: 1
LOAD: 0
Weight path: ./checkpoints
Test Topks: [20]
using bpr loss
===========end===================
use NORMAL distribution initilizer
loading adjacency matrix
successfully loaded...
don't split the matrix
lgn is already to go(dropout:0)
load and save to /content/drive/MyDrive/github/LightGCN_PyTorch/code/checkpoints/lgn-yelp2018-3-64.pth.tar
[TEST]
{'precision': array([0.00027788]), 'recall': array([0.00052693]), 'ndcg': array([0.00044859])}
EPOCH[1/100] loss0.400-|Sample:20.63|
EPOCH[2

In [43]:
# load pretrained model
import model
import dataloader
import world

In [18]:
weight_file = '/content/drive/MyDrive/github/LightGCN_PyTorch/code/checkpoints/lgn-yelp2018-3-64.pth.tar'

In [33]:
#dataset = dataloader.Loader(path+'amazon-book')
path = '/content/drive/MyDrive/github/LightGCN/Data/yelp2018'
dataset = dataloader.Loader(config={'A_split': True, 'A_n_fold': 5}, path=path)


loading [/content/drive/MyDrive/github/LightGCN/Data/yelp2018]
1237259 interactions for training
324147 interactions for testing
yelp2018 Sparsity : 0.0012958757851778647
yelp2018 is ready to go


In [15]:
print(dataset.getSparseGraph()[0].shape)

loading adjacency matrix
torch.Size([13943, 69716])


In [68]:
world.config['pretrain'] = 1

In [94]:
# Save user and item embeddings to separate files

#np.save(user_emb_file, Recmodel.embedding_user.weight.data.cpu().numpy())
#np.save(item_emb_file, Recmodel.embedding_item.weight.data.cpu().numpy())

#user_emb_file = "user_emb.npy"
#item_emb_file = "item_emb.npy"

#world.config['user_emb'] = np.load(user_emb_file)
#world.config['item_emb'] = np.load(item_emb_file)


In [97]:
import numpy as np
import os
# Create a folder to store the embeddings
embedding_folder = "embeddings"
os.makedirs(embedding_folder, exist_ok=True)

# Get the user and item embeddings from the trained model
user_embeddings = Recmodel.embedding_user.weight.data.cpu().numpy()
item_embeddings = Recmodel.embedding_item.weight.data.cpu().numpy()

# Save the embeddings to files
user_emb_file = os.path.join(embedding_folder, "user_embeddings.npy")
item_emb_file = os.path.join(embedding_folder, "item_embeddings.npy")
np.save(user_emb_file, user_embeddings)
np.save(item_emb_file, item_embeddings)


In [98]:
# Load the user and item embeddings from files

user_embeddings = np.load(user_emb_file)
item_embeddings = np.load(item_emb_file)

world.config['user_emb'] = np.load(user_emb_file)
world.config['item_emb'] = np.load(item_emb_file)

In [101]:
Recmodel_load = model.LightGCN(world.config,dataset)

Recmodel_load.load_state_dict(torch.load(weight_file, map_location=torch.device('cpu')))


use pretarined data
loading adjacency matrix
lgn is already to go(dropout:0)


<All keys matched successfully>

In [102]:
world.config

{'bpr_batch_size': 2048,
 'latent_dim_rec': 64,
 'lightGCN_n_layers': 3,
 'dropout': 0,
 'keep_prob': 0.6,
 'A_n_fold': 100,
 'test_u_batch_size': 100,
 'multicore': 0,
 'lr': 0.001,
 'decay': 0.0001,
 'pretrain': 1,
 'A_split': False,
 'bigdata': False,
 'user_emb': array([[-1.4755441 ,  1.9496558 , -0.23540732, ..., -0.9579076 ,
          0.03934332, -1.6575844 ],
        [ 0.41487688, -0.03457997, -0.4186617 , ..., -0.46712428,
         -0.26577732,  0.47366437],
        [-0.8878664 ,  0.00396788, -0.25164598, ...,  0.4763229 ,
         -0.91850364, -0.33945835],
        ...,
        [ 0.2872131 ,  0.393385  , -0.5646815 , ..., -1.3087955 ,
          1.0514622 , -0.3352326 ],
        [ 0.61467314,  0.31270343, -0.9486345 , ..., -2.3770318 ,
          1.9729568 ,  0.5409625 ],
        [-1.659197  ,  0.21919179,  1.666641  , ...,  0.23477043,
          0.9548086 , -0.57835436]], dtype=float32),
 'item_emb': array([[-0.9502222 , -0.9813624 ,  0.65437907, ...,  0.83292323,
         -0.3

In [62]:
Recmodel_load.train()

LightGCN(
  (embedding_user): Embedding(31668, 64)
  (embedding_item): Embedding(38048, 64)
  (f): Sigmoid()
)

In [53]:
#user_embeddings = Recmodel.embedding_user.weight.data
#item_embeddings = Recmodel.embedding_item.weight.data

In [54]:
user_embeddings.shape

torch.Size([31668, 64])

In [55]:
item_embeddings.shape

torch.Size([38048, 64])

In [56]:
Recmodel_load.eval()

LightGCN(
  (embedding_user): Embedding(31668, 64)
  (embedding_item): Embedding(38048, 64)
  (f): Sigmoid()
)

In [104]:
#prediction
users = torch.tensor([15,29,198,477])  # Replace user_id1, user_id2, ... with actual user IDs
ratings = Recmodel.getUsersRating(users)

In [103]:
import Procedure
for epoch in range(world.TRAIN_epochs):
        start = time.time()
        if epoch %10 == 0:
            cprint("[TEST]")
            Procedure.Test(dataset, Recmodel_load, epoch, w, world.config['multicore'])
        output_information = Procedure.BPR_train_original(dataset, Recmodel_load, bpr, epoch, neg_k=Neg_k,w=w)
        print(f'EPOCH[{epoch+1}/{world.TRAIN_epochs}] {output_information}')
        torch.save(Recmodel.state_dict(), weight_file)

[TEST]


TypeError: ignored

In [64]:
#prediction
users = torch.tensor([15,29,198,477])  # Replace user_id1, user_id2, ... with actual user IDs
ratings = Recmodel_load.getUsersRating(users)

TypeError: ignored

In [25]:
ratings

tensor([[0.1171, 0.9999, 1.0000,  ..., 0.5883, 0.7034, 0.6407],
        [1.0000, 0.8707, 0.7832,  ..., 0.1627, 0.2397, 0.4612],
        [1.0000, 0.8773, 0.8904,  ..., 0.2617, 0.3160, 0.5294],
        [0.1289, 0.9112, 0.9485,  ..., 0.7539, 0.5671, 0.7961]],
       device='cuda:0', grad_fn=<SigmoidBackward0>)

In [26]:
top_ratings, top_indices = torch.topk(ratings, 20, dim=1, largest=True)

In [28]:
top_indices

tensor([[  932,  7270,   949,     2,  8129,  4983,  3410,  4048,  4039,  8136,
           947,  3407,  4986,  8119,  4984,   951,  3401,   953,  3405,  4059],
        [  786,    11,   649,    12,  2014,    87,   710,   827,    81,     0,
           769,  1716,   657,   774,   868,  1532,     8,  1540,  4497,  1025],
        [  786,    87,   649,    11,   868,  2014,    81,  4497,   657,     0,
           710,   774,     8,  1532,    82,    12,    93,   721,    13,    15],
        [ 5883,  8106, 12220,  8093,   749,  6351,  6356,  3436,  2645,  6316,
          5886,  9031,  2649,  6335,  2651,  9301,  6348,  8523,  4066,  2660]],
       device='cuda:0')

## GAN